In [1]:
# Import the dependencies
import pandas as pd
import matplotlib as plt
import numpy as np
from citipy import citipy
import requests
from datetime import datetime

# Import API key
from config import weather_api_key
# url = "api.openweathermap.org/data/2.5/weather?q={city name}&appid=" + weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)


In [3]:
# # Add the lats and longs to a list
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
lat_lngs = []
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

620

In [5]:
# list for cities
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | ponta do sol
Processing Record 2 of Set 1 | atuona
Processing Record 3 of Set 1 | korla
Processing Record 4 of Set 1 | carnarvon
Processing Record 5 of Set 1 | rikitea
Processing Record 6 of Set 1 | bambous virieux
Processing Record 7 of Set 1 | ushuaia
Processing Record 8 of Set 1 | maniitsoq
Processing Record 9 of Set 1 | laguna
Processing Record 10 of Set 1 | fortuna
Processing Record 11 of Set 1 | longyearbyen
Processing Record 12 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 13 of Set 1 | port alfred
Processing Record 14 of Set 1 | yar
Processing Record 15 of Set 1 | zhigansk
Processing Record 16 of Set 1 | skeldon
Processing Record 17 of Set 1 | mount gambier
Processing Record 18 of Set 1 | vaini
Processing Record 19 of Set 1 | hithadhoo
Processing Record 20 of Set 1 | ambon
Processing Record 21 of Set 1 | tiznit
Processing Record 22 of Set 1 | jinchang
Processin

In [6]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ponta Do Sol,32.6667,-17.1000,69.71,73,2,2.48,PT,2021-10-16 01:26:57
1,Atuona,-9.8000,-139.0333,77.47,80,3,18.41,PF,2021-10-16 01:26:33
2,Korla,41.7597,86.1469,48.04,40,0,5.97,CN,2021-10-16 01:24:26
3,Carnarvon,-24.8667,113.6333,73.47,56,100,12.66,AU,2021-10-16 01:26:43
4,Rikitea,-23.1203,-134.9692,75.29,71,100,3.74,PF,2021-10-16 01:26:14
5,Bambous Virieux,-20.3428,57.7575,71.85,83,40,12.66,MU,2021-10-16 01:27:33
6,Ushuaia,-54.8000,-68.3000,38.86,87,40,6.91,AR,2021-10-16 01:26:13
7,Maniitsoq,65.4167,-52.9000,36.48,64,0,7.56,GL,2021-10-16 01:27:33
8,Laguna,38.4210,-121.4238,79.30,27,1,4.61,US,2021-10-16 01:22:21
9,Fortuna,40.5982,-124.1573,66.47,69,1,1.01,US,2021-10-16 01:26:23


In [9]:
# Reorder columns
column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[column_order]


In [10]:
# Create the output file CSV
output_data_file = "weather_data/cities.csv"
# Export the City Data into a CSV
city_data_df.to_csv(output_data_file, index_label="City_ID")